In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
j = pd.read_csv('./Data/nfl_draft_prospects.csv')
draft = pd.read_csv('./Data/nfl_draft_profiles.csv')

In [3]:
scout = draft.merge(j,how='left',on='player_id')

In [4]:
scout = scout[['player_name_x','pos_abbr_x','weight_x','height_x','school_x','pos_rk_x','ovr_rk_x','grade_x','text1','text2','text3','text4','draft_year','overall','round','team_abbr']]

In [5]:
scout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12905 entries, 0 to 12904
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  12905 non-null  object 
 1   pos_abbr_x     12905 non-null  object 
 2   weight_x       12905 non-null  int64  
 3   height_x       12905 non-null  float64
 4   school_x       12905 non-null  object 
 5   pos_rk_x       5761 non-null   float64
 6   ovr_rk_x       5100 non-null   float64
 7   grade_x        5767 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     12901 non-null  float64
 13  overall        11343 non-null  float64
 14  round          11343 non-null  float64
 15  team_abbr      11343 non-null  object 
dtypes: float64(7), int64(1), object(8)
memory usage: 1.7+ MB


In [6]:
texts = scout[(scout.text1.notna())|(scout.text2.notna())|(scout.text3.notna())|(scout.text4.notna())]

In [8]:
texts['report'] = None

C:\Users\capta\AppData\Local\Temp\ipykernel_13244\982957188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['report'] = None


In [26]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4883 entries, 7375 to 12904
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  4883 non-null   object 
 1   pos_abbr_x     4883 non-null   object 
 2   weight_x       4883 non-null   int64  
 3   height_x       4883 non-null   float64
 4   school_x       4883 non-null   object 
 5   pos_rk_x       4763 non-null   float64
 6   ovr_rk_x       4165 non-null   float64
 7   grade_x        4769 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     4882 non-null   float64
 13  overall        4040 non-null   float64
 14  round          4040 non-null   float64
 15  team_abbr      4040 non-null   object 
 16  report         2993 non-null   object 
dtypes: float64(7), int64(1), object(9)
memory usage:

In [14]:
texts.loc[(texts.text1.notna()),'report'] = texts.loc[(texts.text1.notna())].text1 

In [ ]:
texts.loc[(texts.text4.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text4.notna())&(texts.text1.isna())].text4

In [28]:
texts.loc[(texts.report.isna())&(texts.text1.isna())] #= texts.loc[(texts.text4.notna())&(texts.text1.isna())].text2

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report
7389,Derrick Johnson,LB,0,0.000,Texas,NaN,NaN,NaN,NaN,The Chiefs wanted Thomas Davis here and actual...,NaN,NaN,2005.0,15.0,1.0,KC,None
7406,Logan Mankins,OG,307,0.000,Fresno State,NaN,NaN,NaN,NaN,The Patriots lost OG Joe Andruzzi to free agen...,NaN,NaN,2005.0,32.0,1.0,NE,None
7420,Matt Roth,DE,313,76.625,Iowa,50.0,NaN,20.0,NaN,The Dolphins signed Vonnie Holliday in free ag...,NaN,NaN,2005.0,46.0,2.0,MIA,None
7499,Todd Herremans,OT,303,78.125,Saginaw Valley,41.0,NaN,30.0,NaN,Green Bay traded the 126th (4th) pick to Phila...,NaN,NaN,2005.0,126.0,4.0,PHI,None
8418,William Hayes,DE,258,74.000,Winston-Salem,31.0,NaN,30.0,NaN,"At 6-foot-2, 258 pounds, Hayes is an undersize...",NaN,NaN,2008.0,103.0,4.0,TEN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12468,Joseph Charlton,P,195,76.500,South Carolina,2.0,202.0,48.0,NaN,Charlton is a right-footed punter who owns the...,NaN,NaN,2020.0,NaN,NaN,NaN,None
12475,Benito Jones,DT,316,73.000,Ole Miss,18.0,219.0,44.0,NaN,"Jones is a three-year starter with good size, ...",NaN,NaN,2020.0,NaN,NaN,NaN,None
12480,Josiah Coatney,DT,308,75.375,Ole Miss,19.0,225.0,42.0,NaN,"Coatney has average size, short arms and below...",NaN,NaN,2020.0,NaN,NaN,NaN,None
12520,Rico Dowdle,RB,213,71.250,South Carolina,29.0,289.0,32.0,NaN,Dowdle is a well-built back with average speed...,NaN,NaN,2020.0,NaN,NaN,NaN,None


In [19]:
missing = texts[texts.report.isna()]
missing.draft_year.value_counts()

2015.0    247
2014.0    243
2009.0    239
2010.0    236
2013.0    236
2012.0    233
2011.0    232
2019.0    132
2020.0     31
2008.0     30
2016.0     26
2005.0      4
2018.0      1
Name: draft_year, dtype: int64

In [30]:
missing[missing.draft_year == 2022]

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report


In [11]:
texts.loc[(texts.text2.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text2.notna())&(texts.text1.isna())].text2 

In [12]:
texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna()),'report'] = texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna())].text3 

In [13]:
texts.loc[(texts.report.isna()),'report'] = texts.loc[(texts.report.isna())].text1

In [14]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4883 entries, 7375 to 12904
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  4883 non-null   object 
 1   pos_abbr_x     4883 non-null   object 
 2   weight_x       4883 non-null   int64  
 3   height_x       4883 non-null   float64
 4   school_x       4883 non-null   object 
 5   pos_rk_x       4763 non-null   float64
 6   ovr_rk_x       4165 non-null   float64
 7   grade_x        4769 non-null   float64
 8   text1          2992 non-null   object 
 9   text2          3848 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     4882 non-null   float64
 13  overall        4040 non-null   float64
 14  round          4040 non-null   float64
 15  team_abbr      4040 non-null   object 
 16  report         4883 non-null   object 
dtypes: float64(7), int64(1), object(9)
memory usage:

In [28]:
texts.reset_index(inplace=True)

In [30]:
texts.drop(['index'],axis=1,inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_16664\3926660189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.drop(['index'],axis=1,inplace=True)


In [42]:
texts.report[0]

"Nephew of Michigan State head coach John L. Smith.  Alex attempted only five passes as a freshman reserve QB in 2002.  He took over the starting job three games into his sophomore season vs. Cal in 2003 and wound up completing 65-percent of his passes for 2,247 yards with 15 touchdowns compared to just three interceptions.   He also rushed for 452 yards in 2003.  As a junior in 2004, Smith started all 12 games, went undefeated as a starter and led the Utes to a Fiesta Bowl win over Pittsburgh.  Prior to declaring himself available for the 2005 NFL draft, Smith completed 66.1-percent of his passes for 2,624 yards with 28 TD's and four INT's, while also rushing for another 563 yards and 10 TD's.  Smith could have used another year at the collegiate level in order to bulk up and improve his overall strength, but his decision makes sense following a nearly perfect junior season and with head coach Urban Meyer leaving for Florida.  Smith still needs to get bigger and he lacks ideal arm str

In [43]:
report = texts.report

In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem import snowball

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\capta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stop_words_list = stopwords.words('english')
stemmer = snowball.EnglishStemmer()

In [64]:
def clean (text, tokenizer, stop_words_list,stemmer):
    lower_text = text.lower()
    token_text = tokenizer.tokenize(lower_text)
    stop_text = [word for word in token_text if word not in stop_words_list]
    stem_text = [stemmer.stem(word) for word in stop_text]
    return stem_text
    

In [66]:
text_data = report.apply(lambda x: clean(x,tokenizer, stop_words_list,stemmer))

In [67]:
text_data

0       [nephew, michigan, state, head, coach, john, s...
1       [brown, play, second, fiddl, carnel, william, ...
2       [edward, father, stanley, play, michigan, nfl,...
3       [benson, draft, los, angel, dodger, 12th, roun...
4       [william, start, two, game, play, nine, 2001, ...
                              ...                        
4878    [jone, big, strong, enough, hold, ground, poin...
4879    [carter, unders, fullback, use, back, type, ls...
4880    [adam, averag, size, top, end, speed, play, ba...
4881    [stiner, interchang, safeti, averag, size, spe...
4882    [imatorbhebh, big, target, averag, speed, good...
Name: report, Length: 4883, dtype: object

In [69]:
report['processed_report'] = text_data

C:\Users\capta\AppData\Local\Temp\ipykernel_16664\1503559796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['processed_report'] = text_data
C:\Users\capta\AppData\Local\Temp\ipykernel_16664\1503559796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['processed_report'] = text_data


In [70]:
report.processed_report

0       [nephew, michigan, state, head, coach, john, s...
1       [brown, play, second, fiddl, carnel, william, ...
2       [edward, father, stanley, play, michigan, nfl,...
3       [benson, draft, los, angel, dodger, 12th, roun...
4       [william, start, two, game, play, nine, 2001, ...
                              ...                        
4878    [jone, big, strong, enough, hold, ground, poin...
4879    [carter, unders, fullback, use, back, type, ls...
4880    [adam, averag, size, top, end, speed, play, ba...
4881    [stiner, interchang, safeti, averag, size, spe...
4882    [imatorbhebh, big, target, averag, speed, good...
Name: report, Length: 4883, dtype: object

In [68]:
cv = CountVectorizer(stop_words='english',min_df=.05,max_df=.9,)
cv.fit_transform(text_data)

AttributeError: 'list' object has no attribute 'lower'

In [61]:
cv.get_feature_names_out()

array(['10', '11', '12', '13', '14', '15', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2015', '2016', '2017', '2019', '2020',
       '40', 'ability', 'adequate', 'alt', 'analysis', 'appeared',
       'april', 'arm', 'arms', 'assets', 'athlete', 'athletic',
       'athleticism', 'average', 'backup', 'balance', 'ball', 'best',
       'better', 'big', 'bit', 'blocker', 'blocks', 'body', 'border',
       'bowl', 'br', 'brings', 'bulk', 'burst', 'career', 'catch',
       'catches', 'caught', 'chance', 'class', 'college', 'com',
       'combination', 'combine', 'comes', 'concerns', 'contribute',
       'corner', 'cover', 'coverage', 'day', 'defender', 'defenders',
       'defense', 'defensive', 'depth', 'develop', 'did', 'does', 'doesn',
       'draft', 'durability', 'early', 'edge', 'effective', 'effort',
       'elite', 'em', 'end', 'espn', 'evaluation', 'excellent',
       'experience', 'explosive', 'field', 'final', 'finished',
       'finishing', 'fit', 'fits', 'flas

In [18]:
#texts.to_parquet('./Data/draft_reports.parquet')